In [17]:
import pandas as pd 
import numpy as np 
from sqlalchemy import create_engine 
import os
from dotenv import load_dotenv
from urllib.parse import quote_plus

In [2]:
db_members = pd.read_csv("tables/members_v3.csv")
db_train = pd.read_csv("tables/train_v2.csv")
db_transactions = pd.read_csv("tables/transactions_v2.csv")

In [3]:
print(db_members.describe())
print("Null Values")
print(db_members.isna().sum())

               city            bd  registered_via  registration_init_time
count  2.339968e+06  2.339968e+06    2.339968e+06            2.339968e+06
mean   8.658488e+00  2.781557e+01    5.252725e+00            2.013124e+07
std    6.627701e+00  1.851081e+01    2.748238e+00            3.030759e+04
min    1.000000e+00 -6.998000e+03    1.000000e+00            2.004033e+07
25%    4.000000e+00  2.000000e+01    3.000000e+00            2.012040e+07
50%    6.000000e+00  2.600000e+01    3.000000e+00            2.014101e+07
75%    1.300000e+01  3.500000e+01    9.000000e+00            2.015082e+07
max    2.200000e+01  1.970000e+03    1.900000e+01            2.017043e+07
Null Values
msno                      0
city                      0
bd                        0
gender                    0
registered_via            0
registration_init_time    0
dtype: int64


In [4]:
db_members.head()

,msno,city,bd,gender,registered_via,registration_init_time
0,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,6,32,female,9,20110915
1,yLkV2gbZ4GLFwqTOXLVHz0VGrMYcgBGgKZ3kj9RiYu8=,4,30,male,9,20110916
2,WH5Jq4mgtfUFXh2yz+HrcTXKS4Oess4k4W3qKolAeb0=,5,34,male,9,20110916
3,tKmbR4X5VXjHmxERrckawEMZ4znVy1lAQIR1vV5rdNk=,5,19,male,9,20110917
4,I0yFvqMoNkM8ZNHb617e1RBzIS/YRKemHO7Wj13EtA0=,13,63,male,9,20110918


In [5]:
print(db_train.describe())
print("Null Values")
print(db_train.isna().sum())

            is_churn
count  970960.000000
mean        0.089942
std         0.286099
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Null Values
msno        0
is_churn    0
dtype: int64


In [6]:
db_transactions.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,++6eU4LsQ3UQ20ILS7d99XK8WbiVgbyYL4FUgzZR134=,32,90,298,298,0,20170131,20170504,0
1,++lvGPJOinuin/8esghpnqdljm6NXS8m8Zwchc7gOeA=,41,30,149,149,1,20150809,20190412,0
2,+/GXNtXWQVfKrEDqYAzcSw2xSPYMKWNj22m+5XkVQZc=,36,30,180,180,1,20170303,20170422,0
3,+/w1UrZwyka4C9oNH3+Q8fUf3fD8R3EwWrx57ODIsqk=,36,30,180,180,1,20170329,20170331,1
4,+00PGzKTYqtnb65mPKPyeHXcZEwqiEzktpQksaaSC3c=,41,30,99,99,1,20170323,20170423,0


In [7]:
print(db_transactions.describe())
print("Null Values")
print(db_transactions.isna().sum())

       payment_method_id  payment_plan_days  plan_list_price  \
count       1.431009e+06       1.431009e+06     1.431009e+06   
mean        3.791835e+01       6.601770e+01     2.817870e+02   
std         4.964805e+00       1.024864e+02     4.351861e+02   
min         2.000000e+00       0.000000e+00     0.000000e+00   
25%         3.600000e+01       3.000000e+01     9.900000e+01   
50%         4.000000e+01       3.000000e+01     1.490000e+02   
75%         4.100000e+01       3.000000e+01     1.490000e+02   
max         4.100000e+01       4.500000e+02     2.000000e+03   

       actual_amount_paid  is_auto_renew  transaction_date  \
count        1.431009e+06   1.431009e+06      1.431009e+06   
mean         2.813172e+02   7.853025e-01      2.016848e+07   
std          4.354200e+02   4.106124e-01      4.858797e+03   
min          0.000000e+00   0.000000e+00      2.015010e+07   
25%          9.900000e+01   1.000000e+00      2.017023e+07   
50%          1.490000e+02   1.000000e+00      2.017

In [8]:
db_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 970960 entries, 0 to 970959
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   msno      970960 non-null  object
 1   is_churn  970960 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 14.8+ MB


 Removing the entries of the train and transactions tables with no matching ID's in the members table and removing the null value rows in members

In [9]:
db_members = db_members.dropna()
print(db_members.isna().sum())
db_members.shape

msno                      0
city                      0
bd                        0
gender                    0
registered_via            0
registration_init_time    0
dtype: int64


(2339968, 6)

### Taking a sample data of 100000 rows

### Filtering the rows of test and transaction based on the presence of foreign key 'msno' in members table

In [10]:
db_members = db_members.sample(100000)
db_train = db_train.sample(100000)
db_transactions = db_transactions.sample(100000)

In [11]:
db_train = db_train[db_train['msno'].isin(db_members['msno'])]
db_train.shape

(1729, 2)

In [12]:
db_transactions = db_transactions[db_transactions['msno'].isin(db_members['msno'])]
db_transactions.shape

(2031, 9)

Python script for changing csv to sql script

In [13]:

def generate_sql(database,output_file,table_name):
    output = output_file
    with open(output, "w", encoding="utf-8") as f:
        for index, row in database.iterrows():
            values = ",".join(
                [ f"'{str(x).replace('\'', '\\\'')}'" if pd.notna(x) else 'Null' for x in row]
            )
            f.write(f"INSERT INTO {table_name} VALUES ({values});\n")

In [14]:
generate_sql(db_members, "queries/members.sql", "members")
generate_sql(db_train, "queries/train.sql", "train")
generate_sql(db_transactions, "queries/transactions.sql", "transactions")

### Importing the main view as a database

In [ ]:
load_dotenv( dotenv_path= "login.env")

username = os.getenv("mysql_username")
password = os.getenv("mysql_password")
host = os.getenv("mysql_host")
port = os.getenv("mysql_port")
database = os.getenv("mysql_database")

password = quote_plus(password)

print(os.getenv("mysql_username"))
print(os.getenv("mysql_host"))
print(os.getenv("mysql_port"))
print(os.getenv("mysql_database"))

root
127.0.0.1
3306
customer churn


In [16]:

engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}")
print(f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}")
main_db = pd.read_sql("select * from main_view", con=engine)
main_db.head()

mysql+pymysql://root:@Aa9208701516@127.0.0.1:3306/customer churn


OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on 'Aa9208701516@127.0.0.1' ([Errno 11003] getaddrinfo failed)")
(Background on this error at: https://sqlalche.me/e/20/e3q8)